In [10]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import Literal

In [11]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv(dotenv_path="../../.env")
api_key = os.getenv("API_KEY")

model = ChatOpenAI(
    model='gpt-4.1',
    temperature=0,
    api_key=api_key
)

In [9]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

url = "https://m.sports.naver.com/wfootball/article/076/0004305450"

loader = WebBaseLoader(web_path=(url,))
document = loader.load()

In [13]:
class SentimentArticle(BaseModel):
    sentiment: Literal['긍정', '부정', '중립'] = Field(description="감성분석 분류")
    score: float = Field(description='감성분석 점수')

parser = PydanticOutputParser(pydantic_object=SentimentArticle)

prompt_template = """
당신은 뉴스 기사의 감성을 분석하는 AI입니다.
아래 뉴스를 읽고 감성을 '긍정', '부정', '중립' 중 하나로 분류하고, 감성 점수를 json 형식으로 출력하세요.
{format_instructions}
뉴스 기사:
{news_article}
"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=['news_article'],
    partial_variables={"format_instructions" : parser.get_format_instructions()}
)

chain = prompt | model | parser

In [14]:
document

[Document(metadata={'source': 'https://m.sports.naver.com/wfootball/article/076/0004305450', 'title': '\'굿바이 SON, 안녕 토트넘\' 손흥민 오피셜 나오나...BBC급 정보력 "LA FC 협상 진전"', 'language': 'ko'}, page_content='\'굿바이 SON, 안녕 토트넘\' 손흥민 오피셜 나오나...BBC급 정보력 "LA FC 협상 진전"NAVER스포츠메뉴홈야구해외야구축구해외축구농구배구N골프일반e스포츠아웃도어NEW뉴스영상일정순위포토홈 바로가기NAVER스포츠마이팀팀 추가응원하는 팀을 구독해보세요!스포츠야구해외야구축구해외축구농구배구N골프일반e스포츠아웃도어콘텐츠오늘의 경기승부예측연재이슈톡대학스포츠랭킹기타고객센터공식 블로그메뉴 닫기본문 바로가기\'굿바이 SON, 안녕 토트넘\' 손흥민 오피셜 나오나...BBC급 정보력 "LA FC 협상 진전"입력2025.07.29. 오전 11:40기사원문공감좋아요0슬퍼요0화나요0팬이에요0후속기사 원해요0텍스트 음성 변환 서비스본문 듣기를 종료하였습니다.글자 크기 변경공유하기<저작권자(c) 연합뉴스, 무단 전재-재배포, AI 학습 및 활용 금지>[스포츠조선 김대식 기자]손흥민의 미국행이 진전되고 있는 것으로 보인다. 토트넘과의 이별이 다가오고 있는 게 느껴진다.현재 손흥민을 영입을 적극적으로 추진하고 있는 구단은 로스엔젤레스(LA) FC다. LA FC가 손흥민에게 접근했다는 소식은 지난 24일(이하 한국시각) 확인됐다. 당시 영국 디 애슬래틱은 \'복수의 소식통에 따르면, LA FC가 토트넘의 손흥민을 여름 이적시장에서 영입하기 위해 강하게 추진하고 있다. 손흥민에 대한 LA FC의 관심은 이미 보도된 바 있지만, LAFC는 향후 몇 주 안에 이 한국 국가대표를 리그로 데려오기 위해 강하게 밀어붙이고 있다\'고 보도했다.<저작권자(c) 연합뉴스, 무단 전재-재배포, AI 학습 및 활용 금지>디 애슬래틱의 보도가 나왔을 때만 해도

In [ ]:
rt = chain.invoke({'news_article' : text})
print(rt.model_dump_json(indent=2))

In [ ]:
search_url = "https://s.search.naver.com/p/newssearch/3/api/tab/more?cluster_rank=98&de=2025.07.29&ds=2025.07.22&eid=&field=0&force_original=&is_dts=0&is_sug_officeid=0&mynews=0&news_office_checked=&nlu_query=&nqx_theme=%7B%22theme%22%3A%7B%22main%22%3A%7B%22name%22%3A%22people_sports%22%2C%22os%22%3A%22175558%22%2C%22source%22%3A%22NLU%22%2C%22pkid%22%3A%221%22%7D%7D%7D&nscs=0&nso=so%3Ar%2Cp%3A1w%2Ca%3Aall&nx_and_query=&nx_search_hlquery=&nx_search_query=&nx_sub_query=&office_category=0&office_section_code=0&office_type=0&pd=1&photo=0&query=%EC%86%90%ED%9D%A5%EB%AF%BC&query_original=&rev=0&service_area=0&sm=tab_smr&sort=0&spq=3&ssc=tab.news.all&start=61"

import requests
rt = requests.get(search_url).json()

bs4.BeautifulSoup(rt['collection'][0]['html']).findAll("a")

from urllib.parse import urlparse, parse_qs
parse_qs(urlparse(search_url).query)

In [ ]:
from urllib.parse import urlparse, parse_qs, urlencode

